In [ ]:
# # upload your kaggle.json file 
# !pip install kaggle
# # !mkdir /root/.kaggle
# # ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle
# ! chmod 600 ~/.kaggle/kaggle.json
# # !cp kaggle.json /root/.kaggle
# #command to get data from kaggle 
# !kaggle competitions download -c birdclef-2022


In [ ]:
# !pip install torchlibrosa

In [1]:
# !unzip -qq birdclef-2022.zip

In [ ]:
import cv2
import audioread
import logging
import gc
import os
import sys
# sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import random
import time
import warnings

import librosa
import numpy as np
import pandas as pd
import soundfile as sf
# import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from contextlib import contextmanager
from joblib import Parallel, delayed
from pathlib import Path
from typing import Optional
from sklearn.model_selection import StratifiedKFold, GroupKFold

# from albumentations.core.transforms_interface import ImageOnlyTransform
from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation
from tqdm import tqdm

# import albumentations as A
# import albumentations.pytorch.transforms as T

import matplotlib.pyplot as plt
# import efficientnet.tfkeras as efn  # Convolutional Neural Network architecture
import IPython.display as ipd
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
# from efficientnet.keras import preprocess_input
# from keras.callbacks.callbacks import ModelCheckpoint, ReduceLROnPlateau
# from sklearn.utils import class_weight
# from tensorflow.keras.layers import Dense, Dropout, Flatten
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# !pip install torchlibrosa


In [8]:
import numpy as np
import pandas as pd
import json
from sklearn import preprocessing
import os
from tqdm import tqdm
import librosa
import tensorflow as tf
import matplotlib.pyplot as plt

In [17]:

# Data:
TRAIN_DIR = './train_audio/'
IMAGES_DIR = './imageeees/'
SAMPLE_RATE = 32000
VAL_SIZE = 0.2

# Data processing:
N_FFT = 2048
HOP_LEN = 512
WIN_FUNC = 'hann'
N_MELS = 224
F_MIN = 0
F_MAX = SAMPLE_RATE / 2

# Learning process:
NAME_MODEL_0 = "model_0_inst.h5"
NAME_MODEL_0_PIC = 'model_0_pic.png'
NAME_MODEL_0_CHECKPOINT = 'model_0_cp.ckpt'
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 32
N_CHANNELS = 3
EPOCHS = 5
CALL_BACKS = [tf.keras.callbacks.ModelCheckpoint(
    filepath=NAME_MODEL_0_CHECKPOINT,
    save_weights_only=True,
    verbose=0
)]

In [18]:

train_metadata = pd.read_csv('./train_metadata.csv')
train_metadata.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename
0,afrsil1,[],"['call', 'flight call']",12.3910,-1.4930,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:00,https://www.xeno-canto.org/125458,afrsil1/XC125458.ogg
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg
2,afrsil1,[],"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:30,https://www.xeno-canto.org/177993,afrsil1/XC177993.ogg
3,afrsil1,[],"['alarm call', 'call']",17.0922,54.2958,Euodice cantans,African Silverbill,Oscar Campbell,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:00,https://www.xeno-canto.org/205893,afrsil1/XC205893.ogg
4,afrsil1,[],['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,Ross Gallardy,Creative Commons Attribution-NonCommercial-Sha...,3.0,16:30,https://www.xeno-canto.org/207431,afrsil1/XC207431.ogg


In [19]:
# Load work classes:
with open('./scored_birds.json', 'r') as f:
    valid_classes = json.load(f)

primary_labels = train_metadata.primary_label

# Encode labels:
encoder = preprocessing.LabelEncoder()
labels = encoder.fit_transform(primary_labels)
labels = np.uint8(labels)

NUM_CLASSES = len(np.unique(labels))

In [20]:
def noise_reduction(y, sr, plot=True, th=0.3):
    from scipy.fft import fft, fftfreq, ifft
    
    SAMPLE_RATE = 1
    DURATION = len(y) / SAMPLE_RATE
    N = int(SAMPLE_RATE * DURATION)

    yf = fft(y)
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(24, 8))
        axes[0].plot(np.arange(len(y)), y)
        axes[0].set_title('Before Time-Domain')
        axes[1].plot(xf, np.abs(yf))
        axes[1].set_title('Before Frequency-Domain')
        plt.show()
    
    # Filtering Low-Pass
    new_yf = yf.copy()
    middle = len(y) / 2
    new_yf[int(middle - len(y) * th):int(middle + len(y) * th)] = 0
    new_y = ifft(new_yf)
    new_y = new_y.real
    
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(24, 8))
        axes[0].plot(np.arange(len(y)), new_y)
        axes[0].set_title('After Time-Domain')
        axes[1].plot(xf, np.abs(new_yf))
        axes[1].set_title('After Frequency-Domain')
        plt.show()
    
    return new_y, sr

In [36]:
df_info = pd.read_csv('info_df.csv')
print(df_info.shape)
df_info.head(3)

(14852, 4)


,fn,len,label,peaks
0,train_audio/oahama/XC511591.ogg,612864,oahama,4.224#16.06#5.072#3.280#1.248#9.040#17.74#12.8...
1,train_audio/oahama/XC27358.ogg,152137,oahama,2.801#0.512#0.512#0.512#0.512#0.512#0.512#0.51...
2,train_audio/oahama/XC454888.ogg,2002860,oahama,41.86#41.04#33.09#19.14#37.57#1.552#27.06#2.36...


In [72]:
# Load audio:
def loadAudio(filename: str) -> np.ndarray:
    # Load audio:
    signal, sr = librosa.load(
        filename,
        sr=SAMPLE_RATE,
        mono=True,
        dtype=np.float32)
    new_y, sr = noise_reduction(signal, sr, plot=False, th=0.3)

    
    
    return new_y




# Convert audio frame into spectrogram:
def createSpectrogram(frame: np.ndarray) -> np.ndarray:
    specgram = librosa.feature.melspectrogram(
        y=frame,
        sr=SAMPLE_RATE,
        n_fft=N_FFT,
        hop_length=HOP_LEN,
        win_length=N_FFT,
        window='hann',
        center=True,
        pad_mode='reflect',
        power=2.0,
        n_mels=N_MELS,
        fmin=F_MIN,
        fmax=F_MAX,
        dtype=np.float32
    )
    specgram = librosa.amplitude_to_db(specgram, ref=np.max)
    
    return specgram


# Save spectrogram as png:
def saveSpectogram(specgram: np.ndarray, filename: str, label: np.uint8, ind: int) -> None:
    bird_name = filename.rsplit('/', 2)[0]
    file_id = filename.rsplit('/', 2)[1].rsplit('.', 2)[0]
    file_name = IMAGES_DIR + 'class_' + str(labels[i]) + '/' + bird_name + '_' + file_id + '_' + str((ind + 1) * 5) + '.png'
    specgram = specgram + 80 # -80 dB -> Min
    specgram = specgram.astype(np.uint8) # 0 - 255 the pixel value
    plt.axis('off')
    plt.imsave(file_name, specgram)
    

# Common function:
def convertAudio(filename: str, label: np.uint8) -> None:
    signal = loadAudio(filename=TRAIN_DIR + filename)
    frames = framing(
        sig=signal,
        sample_rate=SAMPLE_RATE,
        frame_len=5,
        duration_time=librosa.get_duration(
            y=signal,
            sr=SAMPLE_RATE)
    )
    for i in range(frames.shape[0]):
        specgram = createSpectrogram(frame=frames[i])
        saveSpectogram(
            specgram=specgram,
            filename=filename,
            label=label,
            ind=i
        )

In [ ]:
!rm -rf  "./imageeees"


In [75]:
# Create dirs for each class:
os.mkdir(IMAGES_DIR)
for i in range(np.unique(labels).shape[0]):
    os.mkdir(IMAGES_DIR + 'class_' + str(np.unique(labels)[i]) + '/')
    
# Convert audio into spectragrams:
for i in tqdm(range(train_metadata.shape[0])):
    convertAudio(
        filename=train_metadata.filename.iloc[i],
        label=labels[i])

  0%|          | 0/14852 [00:00<?, ?it/s]


ParameterError: Audio data must be of type numpy.ndarray

In [ ]:
# Make a dataset containing the training spectrograms
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    batch_size=BATCH_SIZE,
    validation_split=VAL_SIZE,
    directory=IMAGES_DIR,
    shuffle=True,
    color_mode='rgb',
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    subset="training",
    label_mode='categorical',
    seed=42
)

# Make a dataset containing the validation spectrogram
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    batch_size=BATCH_SIZE,
    validation_split=VAL_SIZE,
    directory=IMAGES_DIR,
    shuffle=True,
    color_mode='rgb',
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    subset="validation",
    label_mode='categorical',
    seed=42
)

In [ ]:
# Function to prepare our datasets for modelling
def prepare(ds, augment=False):
    # Define our one transformation
    rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    flip_and_rotate = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])
    
    # Apply rescale to both datasets and augmentation only to training
    ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    if augment: ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
    return ds

train_dataset = prepare(train_dataset, augment=False)
valid_dataset = prepare(valid_dataset, augment=False)

In [ ]:
def getModel():
    # Create CNN model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(np.unique(labels).shape[0], activation='softmax'))

    # Compile model
    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.RMSprop(),
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ],
    )
    
    return model


def plotMetrics(history):
    metrics = list()
    for key, value in history.history.items():
        metrics.append(key)
        
    for i in range(int(len(metrics) / 2)):
        plt.figure(figsize=(24, 6))
        plt.plot(history.history[metrics[i]], c ="darkblue")
        plt.plot(history.history[metrics[i + int(len(metrics) / 2)]], c ="crimson")
        plt.legend(["Train", "Validation"])
        plt.title("Model" + metrics[i])
        plt.xlabel("Epoch")
        plt.ylabel(metrics[i])
        plt.grid(True, alpha = 0.2)
        plt.show()

In [ ]:
model = getModel()
tf.keras.utils.plot_model(model, NAME_MODEL_0_PIC, show_shapes=True)
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=CALL_BACKS
)
model.save(NAME_MODEL_0)

In [ ]:
plotMetrics(history)


In [ ]:
pred = {
    'row_id': list(),
    'target': list()
}

test_path = './test_soundscapes/'
files = [f.split('.')[0] for f in sorted(os.listdir(test_path))]

birds_path = './scored_birds.json'
with open(birds_path) as bf:
    birds = json.load(bf)
    
for f in files:
    p = test_path + f + '.ogg'
    
    sig = loadAudio(filename=p)
    duration_sig = librosa.get_duration(y=sig, sr=SAMPLE_RATE)
    sig_framed = framing(sig=sig, sample_rate=SAMPLE_RATE, frame_len=5, duration_time=duration_sig)
    
    for i in range(sig_framed.shape[0]):
        # Get prediction:
        specgram = createSpectrogram(frame=sig_framed[i])
        specgram = specgram + 80 # -80 dB -> Min
        specgram = specgram.astype(np.uint8) # 0 - 255 the pixel value

        y_pred = model.predict(specgram)
        predicted_class = np.argmax(y_pred)

        for b in birds:  
            segment_end = (i + 1) * 5
            row_id = f + '_' + b + '_' + str(segment_end)
            pred['row_id'].append(row_id)
            label_inv = encoder.inverse_transform([predicted_class])
            if label_inv[0] == b:
                pred['target'].append(True)
            else:
                pred['target'].append(False)